#Modelo de Machine Learning de Ponta a Ponta

Depois que apresentamos as soluções para melhorar o desempenho da empresa e recuperar uma parte do faturamento, estamos prontos para criar um Modelo de Machine Learning capaz de prever a demanda para os próximos dias, gerando valor para os parceiros e levando informações preciosas para que eles possam se preparar para o dia de trabalho.

In [3]:
#import das bibliotecas
import pandas as pd
import numpy as np

## Análise de Estrutura

## Remoção de Outliers

In [4]:
#Leitura do csv
df_orders = pd.read_csv("orders.csv")

In [5]:
#Corte de outliers
df_orders = df_orders[(df_orders['order_amount'] >= 15) &
          (df_orders['order_amount'] <= 200)]

In [6]:
#info
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333629 entries, 0 to 368998
Data columns (total 29 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   order_id                           333629 non-null  int64  
 1   store_id                           333629 non-null  int64  
 2   channel_id                         333629 non-null  int64  
 3   payment_order_id                   333629 non-null  int64  
 4   delivery_order_id                  333629 non-null  int64  
 5   order_status                       333629 non-null  object 
 6   order_amount                       333629 non-null  float64
 7   order_delivery_fee                 333629 non-null  float64
 8   order_delivery_cost                327275 non-null  float64
 9   order_created_hour                 333629 non-null  int64  
 10  order_created_minute               333629 non-null  int64  
 11  order_created_day                  333629 no

## Novas Features

In [7]:
#converter order_moment_created para data
df_orders['order_moment_created'] = pd.to_datetime(df_orders['order_moment_created'])

C:\AppData\Local\Temp\ipykernel_11288\2250362991.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_orders['order_moment_created'] = pd.to_datetime(df_orders['order_moment_created'])


In [8]:
#nova coluna com o dia da semana
df_orders['day_of_week'] = df_orders['order_moment_created'].dt.day_of_week

In [9]:
#Análise da quantidade de horas que temos pedidos por dia
df_orders['order_created_hour'].value_counts() \
  .reset_index() \
  .sort_values('order_created_hour')

,order_created_hour,count
6,0,25126
12,1,4002
13,2,1003
15,3,713
18,4,267
17,5,417
19,6,260
21,7,214
23,8,124
22,9,126


In [10]:
#criando função por faixa de horário
def faixa_horario(hora):
  if hora >= 0 and hora <= 5:
    return 'madrugada'
  elif hora >= 6 and hora <= 10:
    return 'manha'
  elif hora >= 11 and hora <= 14:
    return 'almoco'
  elif hora >= 15 and hora <= 18:
    return 'tarde'
  else:
    return 'noite'

faixa_horario(19)

'noite'

In [11]:
#aplicando função e criando uma nova coluna com o nome de faixa de horário
df_orders['faixa_horario'] = df_orders['order_created_hour'].apply(faixa_horario)
df_orders['faixa_horario'].value_counts()

faixa_horario
noite        144269
tarde        119598
almoco        37294
madrugada     31528
manha           940
Name: count, dtype: int64

## DataPrep

In [12]:
#Criando um novo dataset de orders_treatment
df_orders_tratamento = df_orders.copy()

In [13]:
#info
df_orders_tratamento.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333629 entries, 0 to 368998
Data columns (total 31 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   order_id                           333629 non-null  int64         
 1   store_id                           333629 non-null  int64         
 2   channel_id                         333629 non-null  int64         
 3   payment_order_id                   333629 non-null  int64         
 4   delivery_order_id                  333629 non-null  int64         
 5   order_status                       333629 non-null  object        
 6   order_amount                       333629 non-null  float64       
 7   order_delivery_fee                 333629 non-null  float64       
 8   order_delivery_cost                327275 non-null  float64       
 9   order_created_hour                 333629 non-null  int64         
 10  order_created_minute     

### Data Cleaning

#### Exclusão de colunas

In [14]:
#após realizar análise das variáveis excluir variável order_moment_delivered
df_orders_tratamento.drop('order_moment_delivered', axis=1, inplace=True)

In [15]:
#definição das colunas que serão excluídas da nossa base
columns_delete = ['payment_order_id',
'delivery_order_id',
'order_created_minute',
'order_created_month',
'order_created_year',
'order_moment_created',
'order_moment_accepted',
'order_moment_ready',
'order_moment_collected',
'order_moment_in_expedition',
'order_moment_delivering',
'order_moment_finished',
'order_delivery_fee',
'order_delivery_cost',
'order_created_hour']

In [16]:
#excluir colunas
df_orders_tratamento.drop(columns_delete, axis=1, inplace=True)

#### Prenchendo valores nulos

In [17]:
#info
df_orders_tratamento.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333629 entries, 0 to 368998
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   order_id                           333629 non-null  int64  
 1   store_id                           333629 non-null  int64  
 2   channel_id                         333629 non-null  int64  
 3   order_status                       333629 non-null  object 
 4   order_amount                       333629 non-null  float64
 5   order_created_day                  333629 non-null  int64  
 6   order_metric_collected_time        290223 non-null  float64
 7   order_metric_paused_time           272423 non-null  float64
 8   order_metric_production_time       312151 non-null  float64
 9   order_metric_walking_time          269553 non-null  float64
 10  order_metric_expediton_speed_time  304793 non-null  float64
 11  order_metric_transit_time          312372 no

In [18]:
#utilização do fillna com a mediana para preencher os valores do order_metric_collected_time
df_orders_tratamento['order_metric_collected_time'] \
  .fillna(df_orders_tratamento['order_metric_collected_time'].median(), inplace=True)

C:\AppData\Local\Temp\ipykernel_11288\1594935980.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  .fillna(df_orders_tratamento['order_metric_collected_time'].median(), inplace=True)


In [19]:
df_orders_tratamento.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333629 entries, 0 to 368998
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   order_id                           333629 non-null  int64  
 1   store_id                           333629 non-null  int64  
 2   channel_id                         333629 non-null  int64  
 3   order_status                       333629 non-null  object 
 4   order_amount                       333629 non-null  float64
 5   order_created_day                  333629 non-null  int64  
 6   order_metric_collected_time        333629 non-null  float64
 7   order_metric_paused_time           272423 non-null  float64
 8   order_metric_production_time       312151 non-null  float64
 9   order_metric_walking_time          269553 non-null  float64
 10  order_metric_expediton_speed_time  304793 non-null  float64
 11  order_metric_transit_time          312372 no

### Criação do dataset final

In [20]:
#Criação do dataset final de grupos
"""df_orders_treatment_group = df_orders_tratamento.groupby(['store_id', 'channel_id', 'order_created_day', 'day_of_week', 'faixa_horario']) \
  .agg({'order_metric_collected_time':'median',
        'order_metric_paused_time':'median',
        'order_metric_production_time':'median',
        'order_metric_walking_time':'median',
        'order_metric_expediton_speed_time':'median',
        'order_metric_transit_time':'median',
        'order_metric_cycle_time':'median',
        'order_id':'count'}) \
  .reset_index() \
  .sort_values('order_id', ascending=False)

df_orders_treatment_group=df_orders_treatment_group.rename(columns = {'order_id':'demanda'})

df_orders_treatment_group.head()"""

"df_orders_treatment_group = df_orders_tratamento.groupby(['store_id', 'channel_id', 'order_created_day', 'day_of_week', 'faixa_horario'])   .agg({'order_metric_collected_time':'median',\n        'order_metric_paused_time':'median',\n        'order_metric_production_time':'median',\n        'order_metric_walking_time':'median',\n        'order_metric_expediton_speed_time':'median',\n        'order_metric_transit_time':'median',\n        'order_metric_cycle_time':'median',\n        'order_id':'count'})   .reset_index()   .sort_values('order_id', ascending=False)\n\ndf_orders_treatment_group=df_orders_treatment_group.rename(columns = {'order_id':'demanda'})\n\ndf_orders_treatment_group.head()"

In [21]:
#info
df_orders_tratamento.info()

<class 'pandas.core.frame.DataFrame'>
Index: 333629 entries, 0 to 368998
Data columns (total 15 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   order_id                           333629 non-null  int64  
 1   store_id                           333629 non-null  int64  
 2   channel_id                         333629 non-null  int64  
 3   order_status                       333629 non-null  object 
 4   order_amount                       333629 non-null  float64
 5   order_created_day                  333629 non-null  int64  
 6   order_metric_collected_time        333629 non-null  float64
 7   order_metric_paused_time           272423 non-null  float64
 8   order_metric_production_time       312151 non-null  float64
 9   order_metric_walking_time          269553 non-null  float64
 10  order_metric_expediton_speed_time  304793 non-null  float64
 11  order_metric_transit_time          312372 no

### Criando um conjunto de testes

Antes de seguir adiante, vamos precisar criar um conjunto de teste, colocá-lo de lado e nunca checá-lo. <br/>

Quando estimamos o erro de generalização utilizando o conjunto de teste, sua estimativa será muito otimista e será lançado um sistema que não funcionará tão bem quanto o esperado.  <br/>

Isso é chamado de **data snooping bias.** <br/>

O Scikit-Learn fornece algumas funções para dividir conjuntos de dados em vários subconjuntos de diversas maneiras. A função mais simples é train_test_split.

In [22]:
#import do train_test_split
from sklearn.model_selection import train_test_split

In [23]:
#Separa a base de treino e teste
train_set, test_set = train_test_split(df_orders_tratamento,
                                      test_size=0.2,
                                      random_state=42)

df_orders_train = train_set.copy()

In [24]:
#Separa a Label principal
df_orders_treatment_label = df_orders_train[['order_status']].copy()

In [25]:
#Remove a Label da base
df_orders_treatment = df_orders_train.drop('order_status', axis=1)

In [26]:
#Seleciona as variáveis float64
df_float_64 = df_orders_treatment.select_dtypes(np.float64).copy()

In [27]:
#Import do SimpleImputer
from sklearn.impute import SimpleImputer

In [28]:
#Adicionando estratégia da mediana no imputer
imputer = SimpleImputer(strategy='median')

In [29]:
#Treina o imputer
imputer.fit(df_float_64)

SimpleImputer(strategy='median')

In [30]:
#Cria nova matriz preenchendo os valores nulos com o imputer
float_vars = imputer.transform(df_float_64)
#float_vars

In [31]:
#Cria novo dataset para apresentar os valores
df_64 = pd.DataFrame(float_vars, columns=df_float_64.columns,
                     index=df_float_64.index)
df_64.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266903 entries, 303534 to 130883
Data columns (total 8 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   order_amount                       266903 non-null  float64
 1   order_metric_collected_time        266903 non-null  float64
 2   order_metric_paused_time           266903 non-null  float64
 3   order_metric_production_time       266903 non-null  float64
 4   order_metric_walking_time          266903 non-null  float64
 5   order_metric_expediton_speed_time  266903 non-null  float64
 6   order_metric_transit_time          266903 non-null  float64
 7   order_metric_cycle_time            266903 non-null  float64
dtypes: float64(8)
memory usage: 18.3 MB


## Seleção Final

## Variáveis Categóricas

In [32]:
#info
df_orders_treatment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 266903 entries, 303534 to 130883
Data columns (total 14 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   order_id                           266903 non-null  int64  
 1   store_id                           266903 non-null  int64  
 2   channel_id                         266903 non-null  int64  
 3   order_amount                       266903 non-null  float64
 4   order_created_day                  266903 non-null  int64  
 5   order_metric_collected_time        266903 non-null  float64
 6   order_metric_paused_time           218000 non-null  float64
 7   order_metric_production_time       249801 non-null  float64
 8   order_metric_walking_time          215700 non-null  float64
 9   order_metric_expediton_speed_time  243894 non-null  float64
 10  order_metric_transit_time          249940 non-null  float64
 11  order_metric_cycle_time            2565

In [33]:
#seleção de day_of_week e faixa_horario
df_orders_treatment[['day_of_week', 'faixa_horario']]

,day_of_week,faixa_horario
303534,6,tarde
134090,2,tarde
38666,6,tarde
208576,4,noite
118311,3,tarde
...,...,...
128685,0,madrugada
286351,2,almoco
141575,5,almoco
158005,4,madrugada


In [34]:
#value_counts faixa de horario
df_orders_treatment['faixa_horario'].value_counts()

faixa_horario
noite        115629
tarde         95665
almoco        29638
madrugada     25219
manha           752
Name: count, dtype: int64

In [35]:
#novo dataset com a seleção das variaveis numéricas categóricas
df_orders_treatment_cat_num = df_orders_treatment[['store_id',
                                                   'channel_id',
                                                   'order_created_day',
                                                   'day_of_week']].copy()

In [36]:
#novo dataset com a seleção das variáveis categóricas
df_orders_treatment_cat = df_orders_treatment[['faixa_horario']].copy()

In [37]:
#import do OrdinalEncoder e do OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

### Ordinal Encoder
A maioria dos algoritmos de Aprendizado de Máquina prefere trabalhar com números, então vamos converter as categorias de texto para números. Para tanto, podemos utilizar o método OrdinalEncoder(), que mapeia cada categoria para um número inteiro diferente.

In [38]:
#tratamento com o OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
df_orders_treatment_cat_encoded = ordinal_encoder.fit_transform(df_orders_treatment_cat)
#df_orders_treatment_cat_encoded[:10]
ordinal_encoder.categories_

[array(['almoco', 'madrugada', 'manha', 'noite', 'tarde'], dtype=object)]

### OneHotEncoder
O OrdinalEncoder pega a quantidade de atributos e converte em números, porém transformando puramente em números ele cria uma diferença de valores entre os números. As categorias 0 e 1 transformadas tem uma distância semelhante, não podemos dizer o mesmo para as categorias 0 e 2, os algoritmos de ML enxergarão essa escala como uma diferença significa entre os dados. <br/>
A utilização do OneHotEncoder é melhor aproveitada para esses casos. Ela cria novos atributos de acordo com a quantidade de atributos com 0 e 1.


In [39]:
#tratamento da categoria com o OneHotEncoder
cat_encoder = OneHotEncoder()
df_orders_treatment_cat_1hot = cat_encoder.fit_transform(df_orders_treatment_cat)
cat_encoder.categories_

[array(['almoco', 'madrugada', 'manha', 'noite', 'tarde'], dtype=object)]

In [40]:
#Visuaulização das categorias em array
df_orders_treatment_cat_1hot.toarray()

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [41]:
#Visualização das categorias
cat_encoder.categories_

[array(['almoco', 'madrugada', 'manha', 'noite', 'tarde'], dtype=object)]

## Escalonando nossos dados

In [42]:
#visualização do df_float64
df_64.head()

,order_amount,order_metric_collected_time,order_metric_paused_time,order_metric_production_time,order_metric_walking_time,order_metric_expediton_speed_time,order_metric_transit_time,order_metric_cycle_time
303534,78.99,1.60,2.43,1316.65,3.50,547.28,922.62,2786.53
134090,35.00,3.95,3.82,8.63,4.37,8.20,9.15,25.97
38666,21.60,2.07,1.60,5.92,6.80,8.40,8.30,22.63
208576,162.20,0.17,2.43,12.23,5.82,6.45,16.50,111.25
118311,25.90,0.95,10.95,1.95,0.97,11.92,10.63,24.50


In [43]:
#import do MinMaxScaler e do StandardScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler

### MinMaxScaler
O escalonamento min-max (muitas pessoas chamam de normalização) é bastante simples: os valores são deslocados e redimensionados para que acabem variando de 0 a 1. Ele subtrai o valor mínimo e divide pelo máximo menos o mínimo. O Scikit-Learn fornece um transformador chamado MinMaxScaler para isso. Ele possui um hiper parâmetro feature_range que permite alterar o intervalo se não quiser 0-1 por algum motivo.

In [44]:
#Tratamento com MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df_64)
df_float64_transform = scaler.transform(df_64)

In [45]:
#Visualiza transformação
df_float64_transform

array([[3.45891892e-01, 1.49619615e-03, 1.89103938e-01, ...,
        4.67701224e-02, 8.43166725e-03, 2.50114635e-02],
       [1.08108108e-01, 2.64149288e-03, 1.89200194e-01, ...,
        7.69688933e-04, 1.08878589e-04, 2.30167944e-04],
       [3.56756757e-02, 1.72525550e-03, 1.89046461e-01, ...,
        7.86755206e-04, 1.01134087e-04, 2.00185068e-04],
       ...,
       [4.02648649e-01, 1.49619615e-03, 1.89103938e-01, ...,
        6.20359040e-04, 1.75845754e-04, 3.27306177e-02],
       [1.07567568e-01, 1.49619615e-03, 1.89103938e-01, ...,
        6.20359040e-04, 1.75845754e-04, 3.68681648e-04],
       [1.31891892e-01, 9.69847018e-04, 1.89183574e-01, ...,
        4.24096895e-04, 1.19356445e-04, 2.59791743e-04]])

### StandardScaler
A padronização é bem diferente: em primeiro lugar ela subtrai o valor médio (assim os valores padronizados sempre têm média zero) e, em seguida, divide pela variância, de modo que a distribuição resultante tenha variância unitária.
Ao contrário do escalonamento min-max, a padronização não vincula valores a um intervalo específico, o que pode ser um problema para alguns algoritmos.
No entanto, a padronização é muito menos afetada por outliers.
O Scikit-Learn fornece um transformador para padronização chamado StandardScaler.

In [46]:
#Tratamento com o SatandardScaler
standard = StandardScaler()
standard.fit(df_64)
df_float64_transform_standard = standard.transform(df_64)

In [47]:
#Visualiza transformação
df_float64_transform_standard

array([[ 0.03734244, -0.07903766, -0.08136245, ...,  7.3095765 ,
         1.22373179,  2.31821102],
       [-0.93265252,  0.14340598, -0.05452367, ..., -0.09993537,
        -0.04984757, -0.10039227],
       [-1.22812722, -0.03454893, -0.09738848, ..., -0.09718643,
        -0.05103266, -0.10331854],
       ...,
       [ 0.26887112, -0.07903766, -0.08136245, ..., -0.12398866,
        -0.03960005,  3.07158453],
       [-0.93485755, -0.07903766, -0.08136245, ..., -0.12398866,
        -0.03960005, -0.08687362],
       [-0.83563097, -0.18126708, -0.0591577 , ..., -0.15560155,
        -0.04824422, -0.09750105]])

**Quando usar cada um?**<br/>
- MinMax Scaling é indicado quando a distribuição dos dados não segue uma gaussiana ou quando você sabe que as variáveis devem estar em um intervalo específico (ex.: redes neurais com funções de ativação como sigmoid).
- Standard Scaling é mais indicado quando os dados têm uma distribuição aproximadamente normal e é necessário manter essa forma de distribuição.

## Pipeline

In [48]:
#import do Pipeline
from sklearn.pipeline import Pipeline

Existem muitas etapas de transformação de dados que precisam ser executadas na ordem correta. Felizmente, o Scikit-Learn fornece a classe Pipeline para ajudar tais sequências de transformações.

O construtor Pipeline se vale de uma lista de pares de nome/estimador que definem uma sequência de etapas. Todos, exceto o último estimador, devem ser transformadores (ou seja, eles devem ter um método fit_transform()).


### Pipeline Numérico

In [49]:
#Criação do novo pipeline com o Imputer e StandardScaler
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])

In [50]:
#Criação de um novo dataset com o pipline numérico
orders_num_tr = num_pipeline.fit_transform(df_float_64)

In [51]:
#Apresentação do Shape da transformação após o pipeline
orders_num_tr.shape

(266903, 8)

### Pipeline Categório + Full Pipeline

In [52]:
#Import do column transform
from sklearn.compose import ColumnTransformer

In [53]:
#Seleção das variáveis numéricas, categóricas numéricas e categóricas.
num_attr = list(df_float_64.columns)
cat_num_attr = list(df_orders_treatment_cat_num.columns)
cat_attr = list(df_orders_treatment_cat.columns)

In [54]:
#Criação do Full Pipeline
full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attr),
    ('cat_num', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_num_attr),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_attr)
])

In [55]:
#Transformação dos dados através do full pipeline
df_orders_prepared = full_pipeline.fit_transform(df_orders_treatment)
df_orders_prepared

array([[ 0.03734244, -0.07903766, -0.08136245, ...,  0.        ,
         0.        ,  1.        ],
       [-0.93265252,  0.14340598, -0.05452367, ...,  0.        ,
         0.        ,  1.        ],
       [-1.22812722, -0.03454893, -0.09738848, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.26887112, -0.07903766, -0.08136245, ...,  0.        ,
         0.        ,  0.        ],
       [-0.93485755, -0.07903766, -0.08136245, ...,  0.        ,
         0.        ,  0.        ],
       [-0.83563097, -0.18126708, -0.0591577 , ...,  0.        ,
         0.        ,  0.        ]])

In [56]:
df_orders_treatment_label

,order_status
303534,FINISHED
134090,FINISHED
38666,FINISHED
208576,FINISHED
118311,FINISHED
...,...
128685,FINISHED
286351,FINISHED
141575,FINISHED
158005,CANCELED


#Classificação

##SGDClassifier

O SGDClassifier (Stochastic Gradient Descent Classifier) é um algoritmo de aprendizado supervisionado, principalmente usado para tarefas de classificação, mas também pode ser aplicado para regressão. Ele é parte da biblioteca scikit-learn e é especialmente útil em problemas de aprendizado com grandes volumes de dados, pois realiza atualizações de parâmetros gradualmente em vez de carregar todo o conjunto de dados na memória de uma só vez.

### **SGDClassifier: Fórmulas e Explicação**

O `SGDClassifier` aplica o Gradiente Estocástico para minimizar uma função de custo associada ao modelo escolhido, como a **SVM Linear** ou a **Regressão Logística**.

### 1. SVM Linear (Função de Custo Hinge)

Para uma SVM Linear, o `SGDClassifier` minimiza a função de custo **hinge**, que maximiza a margem entre as classes:

$$
L(w, b) = \frac{1}{n} \sum_{i=1}^n \max(0, 1 - y_i (w \cdot x_i + b)) + \frac{\alpha}{2} \|w\|^2
$$

Onde:
- $( w $): vetor de pesos (ou coeficientes) do modelo.
- $( b $): bias (intercepto).
- $( x_i $): vetor de características da amostra $( i $).
- $( y_i $): rótulo verdadeiro da amostra $( i $) (normalmente +1 ou -1 para SVM).
- $( n $): número de amostras no lote.
- $( \alpha $): parâmetro de regularização, que controla o impacto da penalidade $( \|w\|^2 $) para evitar overfitting.

A função hinge aplica uma penalidade apenas para amostras que estão incorretamente classificadas ou muito próximas da margem de decisão.


In [57]:
#import do SGDCLassifier
from sklearn.linear_model import SGDClassifier

#Treinamento do Modelo
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(df_orders_prepared, df_orders_treatment_label)

C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(random_state=42)

In [58]:
#Seleciona os 10 primeiros registros de dados e labels
dados = df_orders_treatment.iloc[:10]
labels = df_orders_treatment_label.iloc[:10]

#Passa pelo full pipeline
dados_preparados = full_pipeline.transform(dados)

In [59]:
#Realiza a predição dos valores
sgd_clf.predict(dados_preparados)

array(['FINISHED', 'FINISHED', 'FINISHED', 'FINISHED', 'FINISHED',
       'FINISHED', 'FINISHED', 'FINISHED', 'FINISHED', 'FINISHED'],
      dtype='<U8')

In [60]:
#Verifica as Labels
labels.values

array([['FINISHED'],
       ['FINISHED'],
       ['FINISHED'],
       ['FINISHED'],
       ['FINISHED'],
       ['FINISHED'],
       ['FINISHED'],
       ['FINISHED'],
       ['FINISHED'],
       ['FINISHED']], dtype=object)

### Validação Cruzada (cross_val_score)

Utilizamos o cross_val_score em problemas de classificação para obter uma avaliação mais robusta e confiável do desempenho do modelo. Ele realiza a validação cruzada, que divide o conjunto de dados em múltiplos subconjuntos (ou folds) e testa o modelo em diferentes partições dos dados, ajudando a verificar a capacidade de generalização do modelo para dados novos.

**A Validação Cruzada Reduz o Overfitting**: Em vez de treinar e testar o modelo em uma única divisão dos dados, a validação cruzada usa várias divisões, minimizando o risco de overfitting. Com isso, evitamos ajustar o modelo apenas a uma fração específica dos dados.



In [61]:
#import do cross_val_score
from sklearn.model_selection import cross_val_score
#Realiza o cross validation com 3 folds e orientado pela "accuracy"
cross_val_score(sgd_clf, df_orders_prepared, df_orders_treatment_label, cv=3, scoring='accuracy')

C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0.95677097, 0.95629889, 0.95557903])

### Validação Cruzada de Predição

O cross_val_predict também utiliza a validação cruzada, mas, em vez de retornar uma pontuação média como o cross_val_score, ele gera diretamente as predições do modelo para cada amostra do conjunto de dados.

Em resumo, o cross_val_predict é útil quando queremos:

**Obter Predições de Validação Cruzada:** Ele retorna uma matriz com as predições do modelo para cada amostra no conjunto de dados, permitindo que analisemos o desempenho do modelo em nível de amostra.

**Análise Detalhada de Métricas:** Como temos as predições para cada amostra, podemos calcular métricas como precisão, recall, F1-score, curva ROC e AUC, utilizando todas as amostras do conjunto de dados como se fossem dados de teste, sem afetar o processo de treino.

**Redução de Overfitting em Métricas de Avaliação:** Calculando as métricas a partir das predições feitas por cross_val_predict, conseguimos uma avaliação mais próxima de um cenário real, onde cada predição foi feita com dados de treino diferentes dos dados de teste, evitando o uso de um conjunto fixo de treino e teste.

In [62]:
#import do cross_val_predict
from sklearn.model_selection import cross_val_predict
#realiza a predição através do cross_val_predict
y_train_pred = cross_val_predict(sgd_clf, df_orders_prepared, df_orders_treatment_label, cv=3)

C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Matriz de Confusão

A matriz de confusão é uma ferramenta muito útil para avaliar o desempenho de um modelo de classificação. Ela mostra, em uma tabela, as predições feitas pelo modelo versus os valores reais, ajudando a identificar erros específicos em cada classe e a entender onde o modelo está acertando ou errando.

**Estrutura da Matriz de Confusão**
Para um problema de classificação binária, a matriz de confusão é normalmente organizada da seguinte forma:


\begin{array}{|c|c|c|}
\hline
 & \text{Previsão Positiva} & \text{Previsão Negativa} \\
\hline
\text{Classe Positiva} & \text{True Positive (TP)} & \text{False Negative (FN)} \\
\hline
\text{Classe Negativa} & \text{False Positive (FP)} & \text{True Negative (TN)} \\
\hline
\end{array}

### Componentes da Matriz de Confusão

1. **True Positive (TP)**: Amostras corretamente classificadas como positivas.
2. **False Positive (FP)**: Amostras negativas incorretamente classificadas como positivas (**falso alarme**).
3. **True Negative (TN)**: Amostras corretamente classificadas como negativas.
4. **False Negative (FN)**: Amostras positivas incorretamente classificadas como negativas (**falta de detecção**).

#### Métricas Derivadas da Matriz de Confusão

A partir dos valores de TP, FP, TN e FN, podemos calcular várias métricas para avaliar o modelo:

1. **Acurácia**: Percentual de predições corretas em relação ao total de amostras.
   $
   \text{Acurácia} = \frac{TP + TN}{TP + TN + FP + FN}
   $

2. **Precisão**: Percentual de predições positivas que são realmente positivas.
   $
   \text{Precisão} = \frac{TP}{TP + FP}
   $

3. **Recall (ou Sensibilidade)**: Percentual de amostras positivas corretamente identificadas.
   $
   \text{Recall} = \frac{TP}{TP + FN}
   $

4. **F1-Score**: Média harmônica entre precisão e recall, útil quando as classes são desbalanceadas.
   $
   F1 = 2 \times \frac{\text{Precisão} \times \text{Recall}}{\text{Precisão} + \text{Recall}}
   $


In [63]:
#import da matriz de confusão
from sklearn.metrics import confusion_matrix
#matriz de confusão
confusion_matrix(df_orders_treatment_label, y_train_pred)

array([[    13,  11237],
       [   449, 255204]], dtype=int64)

### Classification Report

O `classification_report` do `sklearn` gera um relatório com métricas de desempenho de um modelo de classificação para cada classe, facilitando a análise e comparação do desempenho entre classes.

## Principais Métricas do `classification_report`

1. **Acurácia**: Percentual de predições corretas em relação ao total de amostras.
   $
   \text{Acurácia} = \frac{TP + TN}{TP + TN + FP + FN}
   $


2. **Precisão (Precision)**:
   - A precisão indica a porcentagem de predições positivas que são realmente positivas.
   - Fórmula:
     $
      \text{Precisão} = \frac{TP}{TP + FP}
      $

3. **Recall (Sensibilidade)**:
   - O recall mede a capacidade do modelo de identificar corretamente todas as amostras da classe positiva.
   - Fórmula:
     $
   \text{Recall} = \frac{TP}{TP + FN}
   $

4. **F1-Score**:
   - O F1-Score é a média harmônica entre precisão e recall, útil quando temos classes desbalanceadas.
   - Fórmula:
     $
   F1 = 2 \times \frac{\text{Precisão} \times \text{Recall}}{\text{Precisão} + \text{Recall}}
   $

5. **Suporte (Support)**:
   - O suporte é o número de ocorrências reais de cada classe no conjunto de dados, ajudando a avaliar o impacto de cada classe no cálculo das métricas.


In [64]:
#import do classification_report
from sklearn.metrics import classification_report
#realizada o classification report
report = classification_report(df_orders_treatment_label, y_train_pred)
print(report)

              precision    recall  f1-score   support

    CANCELED       0.03      0.00      0.00     11250
    FINISHED       0.96      1.00      0.98    255653

    accuracy                           0.96    266903
   macro avg       0.49      0.50      0.49    266903
weighted avg       0.92      0.96      0.94    266903



In [65]:
#Analisa a porcentagem entre status
df_orders_treatment_label['order_status'].value_counts()

order_status
FINISHED    255653
CANCELED     11250
Name: count, dtype: int64

### Desbalanceamento entre Classes

O desbalanceamento de classes pode afetar negativamente o desempenho de modelos de machine learning, pois o modelo pode aprender a favorecer a classe majoritária, ignorando as amostras da classe minoritária. Essas são umas das principais estratégias para lidar com classes desbalanceadas:

#### Oversampling
O oversampling consiste em aumentar a quantidade de instâncias da classe minoritária, de forma que ela tenha uma representação mais equilibrada em relação à classe majoritária. O método mais simples de oversampling é a duplicação de dados da classe minoritária, mas isso pode levar a overfitting.

#### Undersampling
O undersampling é o processo de reduzir a quantidade de instâncias da classe majoritária, diminuindo o número de amostras para equilibrá-lo com a classe minoritária. A técnica mais simples de undersampling é a remoção aleatória de instâncias da classe majoritária. No entanto, isso pode resultar na perda de informações importantes, principalmente se o conjunto de dados for pequeno.

## SMOTE (Synthetic Minority Over-sampling Technique)
O SMOTE é uma técnica de oversampling que cria novas instâncias sintéticas da classe minoritária em vez de duplicar instâncias. Ele funciona da seguinte maneira:

Para cada instância da classe minoritária, o SMOTE seleciona uma ou mais instâncias vizinhas próximas.
Novas instâncias são geradas interpolando os atributos entre a instância original e a vizinha selecionada.
Esse processo ajuda a evitar o overfitting, pois as novas amostras são únicas e não cópias diretas das amostras existentes.




In [66]:
#import do SMOTE
from imblearn.over_sampling import SMOTE


In [67]:
# criando uma instância do SMOTE
smote = SMOTE()

# balanceando os dados
x_resampled, y_resampled = smote.fit_resample(df_orders_prepared, df_orders_treatment_label)

In [68]:
#Análise entre status
y_resampled['order_status'].value_counts()

In [70]:
#Realizando o treino novamente com o SGDClassifier
sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(x_resampled, y_resampled)

C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(random_state=42)

In [72]:
#Realizando a validação cruzada orientada a previsão pelo resampling do SMOTE
y_train_pred = cross_val_predict(sgd_clf, x_resampled, y_resampled, cv=3)

C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [73]:
#Análise do classification_report
from sklearn.metrics import classification_report
report = classification_report(y_resampled, y_train_pred)
print(report)

              precision    recall  f1-score   support

    CANCELED       0.59      0.64      0.61    255653
    FINISHED       0.60      0.55      0.57    255653

    accuracy                           0.59    511306
   macro avg       0.59      0.59      0.59    511306
weighted avg       0.59      0.59      0.59    511306



## RandomForestClassifier

In [74]:
#Import do RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [76]:
#Treina o modelo no RandomForestClassifier
forest_clf = RandomForestClassifier(n_estimators=10, random_state=42)
forest_clf.fit(x_resampled, y_resampled)

C:\Program Files\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(n_estimators=10, random_state=42)

In [78]:
#Realiza a Validação Cruzada
y_train_pred = cross_val_predict(forest_clf, x_resampled, y_resampled, cv=3)

C:\Program Files\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Program Files\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Program Files\Python312\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [79]:
#Print do Classification Report
from sklearn.metrics import classification_report
report = classification_report(y_resampled, y_train_pred)
print(report)

              precision    recall  f1-score   support

    CANCELED       1.00      0.99      0.99    255653
    FINISHED       0.99      1.00      0.99    255653

    accuracy                           0.99    511306
   macro avg       0.99      0.99      0.99    511306
weighted avg       0.99      0.99      0.99    511306



### Modelo de Classificação Final

In [80]:
#Escolhe o modelo
final_model = forest_clf

In [ ]:
#Seleciona o dataset de teste


In [ ]:
#Analise do classification_report
from sklearn.metrics import classification_report
report = classification_report(y_test, final_predictions)
print(report)